<a href="https://colab.research.google.com/github/JoshuaLee0816/diffuserlite.github.io/blob/main/01_DiffuserLite_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

# === 1. 設定你的 GitHub 資訊 ===
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')# 建議之後更換
GITHUB_USER = "JoshuaLee0816"
GITHUB_REPO = "diffuserlite.github.io"

import os
os.chdir('/content')

# === 2. Clone 你的倉庫作為「唯一」工作目錄 ===
!rm -rf /content/{GITHUB_REPO}
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{GITHUB_REPO}.git

# 進入倉庫
%cd /content/{GITHUB_REPO}

# === 3. 安裝環境（在你的 Repo 目錄下安裝） ===
!pip install -e . -q  # 以可編輯模式安裝目前的倉庫
!pip install git+https://github.com/Farama-Foundation/D4RL.git --ignore-requires-python -q
!pip install "numpy>=1.26.0,<2.0.0" -q
!pip install --upgrade huggingface_hub -q  # Hugging Face 上傳用

# MuJoCo 210 安裝
!mkdir -p /root/.mujoco
!wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O /tmp/mujoco210.tar.gz
!tar -xzf /tmp/mujoco210.tar.gz -C /root/.mujoco/
!apt-get install -qq -y libosmesa6-dev libgl1-mesa-glx libglfw3 patchelf > /dev/null 2>&1

# 設定 Git 身份
!git config --global user.email "310274movie@gmail.com"
!git config --global user.name "JoshuaLee"

print("✅ 環境已在你的 Repo 中安裝完成！現在所有變動都會記錄在此。")


In [ ]:
# === Hugging Face 登入（只需執行一次）===
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

In [ ]:
%%writefile /content/run.py
import sys, os, types, json, glob, re, time, traceback
import mujoco
import numpy as np
import yaml

# === 強制 unbuffered output ===
sys.stdout = os.fdopen(sys.stdout.fileno(), 'w', buffering=1)
sys.stderr = os.fdopen(sys.stderr.fileno(), 'w', buffering=1)

print("[DEBUG] run.py 開始執行")

shim = types.ModuleType('mujoco_py')

class ShimModel:
    def __init__(self, m):
        self._m = m
    @property
    def actuator_ctrlrange(self):
        return self._m.actuator_ctrlrange.copy()
    @property
    def nq(self): return self._m.nq
    @property
    def nv(self): return self._m.nv
    def __getattr__(self, name):
        return getattr(self._m, name)

class ShimData:
    def __init__(self, d):
        self._d = d
    @property
    def qpos(self): return self._d.qpos
    @qpos.setter
    def qpos(self, v): self._d.qpos[:] = v
    @property
    def qvel(self): return self._d.qvel
    @qvel.setter
    def qvel(self, v): self._d.qvel[:] = v
    @property
    def ctrl(self): return self._d.ctrl
    @ctrl.setter
    def ctrl(self, v): self._d.ctrl[:] = v
    def __getattr__(self, name):
        return getattr(self._d, name)

class ShimSim:
    def __init__(self, model):
        self._m = model._m
        self._d = mujoco.MjData(self._m)
        self.model = model
        self.data = ShimData(self._d)
    def step(self):
        mujoco.mj_step(self._m, self._d)
    def forward(self):
        mujoco.mj_forward(self._m, self._d)
    def get_state(self):
        return type('S',(),{'time':self._d.time,'qpos':self._d.qpos.copy(),'qvel':self._d.qvel.copy(),'act':np.array([]),'udd_state':{}})()
    def set_state(self, s):
        self._d.time = s.time
        self._d.qpos[:] = s.qpos
        self._d.qvel[:] = s.qvel
        mujoco.mj_forward(self._m, self._d)

class MjViewer:
    def __init__(self, sim): pass
    def render(self): pass

shim.load_model_from_path = lambda p: ShimModel(mujoco.MjModel.from_xml_path(p))
shim.MjSim = lambda m: ShimSim(m)
shim.MjViewer = MjViewer
shim.MujocoException = Exception
shim.ignore_mujoco_warnings = type('ctx',(),{'__enter__':lambda s:None,'__exit__':lambda s,*a:None})
shim.__path__ = []
for sub in ['cymj','builder','generated','generated.const']:
    sys.modules[f'mujoco_py.{sub}'] = types.ModuleType(f'mujoco_py.{sub}')
sys.modules['mujoco_py'] = shim

print("[DEBUG] mujoco_py shim 已設定")

# === 設定路徑 ===
os.chdir('/content/diffuserlite.github.io')
os.makedirs('results/figures', exist_ok=True)

# === 讀取環境名稱 ===
with open('configs/diffuserlite/mujoco/mujoco.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 解析 defaults 中的 task
defaults = config.get('defaults', [])
ENV_NAME = 'halfcheetah-medium-expert-v2'  # 預設值
for item in defaults:
    if isinstance(item, dict) and 'task' in item:
        ENV_NAME = item['task']
        break

# 也讀取 mode
MODE = config.get('mode', 'training')

print(f"🎮 訓練環境: {ENV_NAME}")
print(f"📋 訓練模式: {MODE}")
print(f"📁 工作目錄: {os.getcwd()}")
sys.stdout.flush()

# === 從 HuggingFace 下載之前的 loss_log.json（如果存在）===
original_print = print
loss_log = []
loss_log_path = f'results/loss_log_{ENV_NAME}.json'

try:
    from huggingface_hub import hf_hub_download
    repo_id = "JoshuaLee0816/diffuserlite-results"
    local_path = hf_hub_download(repo_id=repo_id, filename=f"loss_log_{ENV_NAME}.json")
    with open(local_path, 'r') as f:
        loss_log = json.load(f)
    original_print(f"✅ 從 HuggingFace 載入 {len(loss_log)} 筆歷史 loss 記錄")
    if loss_log:
        last_step = loss_log[-1].get('gradient_steps', 0)
        original_print(f"   上次訓練到第 {last_step} 步")
except Exception as e:
    original_print(f"⚠️ 無法載入歷史 loss_log: {e}")
    original_print("   將從頭開始記錄")

sys.stdout.flush()

# 儲存到本地
with open(loss_log_path, 'w') as f:
    json.dump(loss_log, f)

# === HuggingFace 上傳函數（帶 rate limit 處理）===
def upload_single_file_with_retry(api, local_path, repo_path, repo_id, max_retries=5):
    for attempt in range(max_retries):
        try:
            api.upload_file(
                path_or_fileobj=local_path,
                path_in_repo=repo_path,
                repo_id=repo_id,
            )
            return True
        except Exception as e:
            error_str = str(e).lower()
            if "rate limit" in error_str or "429" in str(e) or "too many" in error_str:
                wait_time = 60 * (attempt + 1)
                original_print(f"  ⏳ Rate limit! 等待 {wait_time} 秒後重試 ({attempt+1}/{max_retries})...")
                time.sleep(wait_time)
            else:
                if attempt < max_retries - 1:
                    original_print(f"  ⚠️ 上傳失敗: {e}，等待 30 秒後重試...")
                    time.sleep(30)
                else:
                    return False
    return False

def upload_to_hf(files_to_upload, message=""):
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        repo_id = "JoshuaLee0816/diffuserlite-results"
        
        success_count = 0
        for local_path, repo_path in files_to_upload:
            if os.path.exists(local_path):
                if upload_single_file_with_retry(api, local_path, repo_path, repo_id):
                    success_count += 1
                else:
                    original_print(f"  ❌ 上傳失敗: {repo_path}")
                time.sleep(2)
        
        if message and success_count > 0:
            original_print(f"📤 {message} ({success_count}/{len(files_to_upload)} 檔案)")
    except Exception as e:
        original_print(f"⚠️ HuggingFace 上傳失敗: {e}")

def cleanup_old_checkpoints(checkpoint_dir):
    try:
        for prefix in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
            pattern = os.path.join(checkpoint_dir, f'{prefix}_ckpt_*.pt')
            files = glob.glob(pattern)
            numbered_files = [f for f in files if 'latest' not in f]
            for f in numbered_files:
                match = re.search(r'_ckpt_(\d+)\.pt', f)
                if match:
                    step = int(match.group(1))
                    if step % 5000 != 1000:
                        os.remove(f)
    except Exception as e:
        original_print(f"⚠️ 清理舊 checkpoint 失敗: {e}")

def delete_from_hf(files_to_delete):
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        repo_id = "JoshuaLee0816/diffuserlite-results"
        for repo_path in files_to_delete:
            try:
                api.delete_file(path_in_repo=repo_path, repo_id=repo_id)
                time.sleep(1)
            except:
                pass
    except:
        pass

def update_figure():
    import matplotlib.pyplot as plt
    if not loss_log:
        return
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    steps = [d['gradient_steps'] for d in loss_log]
    for key in ['loss0', 'loss1', 'loss2']:
        if key in loss_log[0]:
            values = [d[key] for d in loss_log]
            axes[0].plot(steps, values, label=key, alpha=0.8)
    axes[0].set_xlabel('Gradient Steps')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'Diffusion Model Loss ({ENV_NAME})')
    axes[0].legend()
    axes[0].grid(True)
    if 'invdyn_loss' in loss_log[0]:
        values = [d['invdyn_loss'] for d in loss_log]
        axes[1].plot(steps, values, color='orange', alpha=0.8)
        axes[1].set_xlabel('Gradient Steps')
        axes[1].set_ylabel('Loss')
        axes[1].set_title(f'Inverse Dynamics Loss ({ENV_NAME})')
        axes[1].grid(True)
    plt.tight_layout()
    plt.savefig(f'results/figures/loss_curve_{ENV_NAME}.png', dpi=150)
    plt.close()

# === 攔截 print 來記錄 loss ===
last_upload_step = 0
checkpoint_dir = f'results/diffuserlite_d4rl_mujoco/{ENV_NAME}'
hf_checkpoint_prefix = f'diffuserlite_d4rl_mujoco/{ENV_NAME}'

def custom_print(*args, **kwargs):
    global last_upload_step
    original_print(*args, **kwargs)
    sys.stdout.flush()
    
    if args and isinstance(args[0], dict) and 'gradient_steps' in args[0]:
        current_step = args[0]['gradient_steps']
        loss_log.append(args[0].copy())
        
        with open(loss_log_path, 'w') as f:
            json.dump(loss_log, f)
        
        if current_step - last_upload_step >= 1000:
            update_figure()
            
            files = [
                (loss_log_path, f'loss_log_{ENV_NAME}.json'),
                (f'results/figures/loss_curve_{ENV_NAME}.png', f'figures/loss_curve_{ENV_NAME}.png'),
            ]
            
            for name in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
                ckpt_path = f'{checkpoint_dir}/{name}_ckpt_latest.pt'
                if os.path.exists(ckpt_path):
                    files.append((ckpt_path, f'{hf_checkpoint_prefix}/{name}_ckpt_latest.pt'))
            
            for name in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
                ckpt_path = f'{checkpoint_dir}/{name}_ckpt_{current_step}.pt'
                if os.path.exists(ckpt_path):
                    files.append((ckpt_path, f'{hf_checkpoint_prefix}/{name}_ckpt_{current_step}.pt'))
            
            upload_to_hf(files, f"已上傳 step {current_step} 的 checkpoint")
            
            if current_step % 5000 != 1000 and last_upload_step > 0:
                files_to_delete = []
                for name in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
                    files_to_delete.append(f'{hf_checkpoint_prefix}/{name}_ckpt_{last_upload_step}.pt')
                delete_from_hf(files_to_delete)
            
            cleanup_old_checkpoints(checkpoint_dir)
            last_upload_step = current_step

import builtins
builtins.print = custom_print

# === 在 pipeline 中插入 debug 訊息 ===
original_print("[DEBUG] 準備修改 pipeline 加入 debug 訊息...")

pipeline_path = 'pipelines/diffuserlite_d4rl_mujoco.py'
with open(pipeline_path, 'r') as f:
    pipeline_code = f.read()

# 在關鍵位置插入 debug 訊息
debug_patches = [
    ('def pipeline(args):', 'def pipeline(args):\n    print("[DEBUG] pipeline() 被呼叫")\n    print(f"[DEBUG] args.mode = {args.mode}")\n    print(f"[DEBUG] args.task.env_name = {args.task.env_name}")\n    import sys; sys.stdout.flush()'),
    ('if args.mode == "training":', 'if args.mode == "training":\n        print("[DEBUG] 進入 training 模式")\n        import sys; sys.stdout.flush()'),
    ('for batch in loop_dataloader(dataloader):', 'print("[DEBUG] 開始訓練迴圈...")\n        import sys; sys.stdout.flush()\n        for batch in loop_dataloader(dataloader):'),
    ('env = gym.make(args.task.env_name)', 'print(f"[DEBUG] 正在建立環境: {args.task.env_name}")\n    import sys; sys.stdout.flush()\n    env = gym.make(args.task.env_name)\n    print("[DEBUG] 環境建立成功")'),
    ('dataloader = DataLoader(', 'print("[DEBUG] 正在建立 DataLoader...")\n    import sys; sys.stdout.flush()\n    dataloader = DataLoader('),
]

for old, new in debug_patches:
    if old in pipeline_code:
        pipeline_code = pipeline_code.replace(old, new, 1)
        original_print(f"[DEBUG] 已插入 debug: {old[:30]}...")

# 儲存修改後的 pipeline
with open(pipeline_path, 'w') as f:
    f.write(pipeline_code)

original_print("[DEBUG] pipeline 已修改完成")
sys.stdout.flush()

# === 執行訓練 ===
original_print("🚀 開始執行訓練 pipeline...")
sys.stdout.flush()

try:
    import runpy
    runpy.run_path(pipeline_path, run_name='__main__')
except SystemExit as e:
    if e.code == 0:
        original_print("[DEBUG] pipeline 正常結束 (exit code 0)")
    else:
        original_print(f"\n❌ pipeline 異常結束 (exit code {e.code})")
except Exception as e:
    original_print(f"\n❌ 訓練過程發生錯誤:")
    original_print(traceback.format_exc())
    sys.exit(1)

# === 訓練結束後最終上傳 ===
original_print("\n" + "="*50)
original_print(f"訓練完成！({ENV_NAME})")
original_print("正在上傳最終結果...")

update_figure()

files = [
    (loss_log_path, f'loss_log_{ENV_NAME}.json'),
    (f'results/figures/loss_curve_{ENV_NAME}.png', f'figures/loss_curve_{ENV_NAME}.png'),
]
for name in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
    ckpt_path = f'{checkpoint_dir}/{name}_ckpt_latest.pt'
    if os.path.exists(ckpt_path):
        files.append((ckpt_path, f'{hf_checkpoint_prefix}/{name}_ckpt_latest.pt'))

upload_to_hf(files, "✅ 最終結果已上傳到 HuggingFace")
original_print("="*50)

# ============================================================
# 斷線恢復：從 HuggingFace 下載 checkpoint
# ============================================================
# 如果 Colab 斷線，重新執行以下步驟：
# 1. 執行「環境安裝」cell
# 2. 執行「HuggingFace 登入」cell
# 3. 執行「從 HuggingFace 下載 checkpoint」cell（下方）
# 4. 執行「設定訓練參數」cell
# 5. 執行「執行訓練」cell

In [ ]:
# === 從 Hugging Face 下載之前的 checkpoint（斷線恢復用）===
from huggingface_hub import hf_hub_download, list_repo_files
import os
import shutil
import json

repo_id = "JoshuaLee0816/diffuserlite-results"
local_results_dir = '/content/diffuserlite.github.io/results/diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2'

# 檢查 Hugging Face 上有沒有之前的 checkpoint
try:
    files = list_repo_files(repo_id)
    
    # 1. 下載 checkpoint
    checkpoint_files = [f for f in files if 'halfcheetah-medium-expert-v2' in f and 'latest' in f and f.endswith('.pt')]
    
    if checkpoint_files:
        print(f"找到 {len(checkpoint_files)} 個 checkpoint 檔案，開始下載...")
        os.makedirs(local_results_dir, exist_ok=True)

        for f in checkpoint_files:
            local_path = hf_hub_download(repo_id=repo_id, filename=f)
            filename = os.path.basename(f)
            dest_path = os.path.join(local_results_dir, filename)
            shutil.copy(local_path, dest_path)
            print(f"  ✅ {filename}")
        print(f"\n✅ 已從 Hugging Face 恢復 checkpoint！")
    else:
        print("⚠️ Hugging Face 上沒有找到 checkpoint，將從頭開始訓練")
    
    # 2. 下載 loss_log.json（用於接續圖表）
    if 'loss_log.json' in files:
        local_path = hf_hub_download(repo_id=repo_id, filename="loss_log.json")
        dest_path = '/content/diffuserlite.github.io/results/loss_log.json'
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copy(local_path, dest_path)
        
        with open(dest_path, 'r') as f:
            loss_log = json.load(f)
        if loss_log:
            last_step = loss_log[-1].get('gradient_steps', 0)
            print(f"✅ 已恢復 loss_log.json（{len(loss_log)} 筆記錄，上次到第 {last_step} 步）")
    else:
        print("⚠️ 沒有找到 loss_log.json，將從頭開始記錄")
        
except Exception as e:
    print(f"⚠️ 無法連接 Hugging Face: {e}")
    print("   將從頭開始訓練")

# 顯示目前本地的 checkpoint
print(f"\n=== 本地 checkpoint 狀態 ===")
if os.path.exists(local_results_dir):
    files = os.listdir(local_results_dir)
    latest_files = [f for f in files if 'latest' in f]
    if latest_files:
        for f in sorted(latest_files):
            size_mb = os.path.getsize(os.path.join(local_results_dir, f)) / (1024*1024)
            print(f"  {f}: {size_mb:.1f} MB")
    else:
        print("  (無 checkpoint)")
else:
    print("  (目錄不存在)")

In [ ]:
# === 階段 1：設定正式訓練參數 ===
# 訓練 100,000 步，每 1000 步存 checkpoint

!sed -i 's/save_interval: [0-9]*/save_interval: 1000/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/diffusion_gradient_steps: [0-9]*/diffusion_gradient_steps: 100000/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/invdyn_gradient_steps: [0-9]*/invdyn_gradient_steps: 100000/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/log_interval: [0-9]*/log_interval: 100/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml

# 確認修改結果
!echo "=== 正式訓練參數（100K 步）==="
!grep -E "save_interval|diffusion_gradient_steps|invdyn_gradient_steps|log_interval|device:" /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml

# ============================================================
# 環境選擇（三選一執行）
# ============================================================
# 預設是 halfcheetah，如果要跑其他環境，執行對應的 cell
# 
# | 環境 | 論文 R1 分數 |
# |------|-------------|
# | halfcheetah-medium-expert-v2 | 91.9 |
# | walker2d-medium-expert-v2 | 109.1 |
# | hopper-medium-expert-v2 | 103.3 |

In [ ]:
# === 選擇環境：HalfCheetah（預設）===
ENV_NAME = "halfcheetah-medium-expert-v2"

import subprocess
subprocess.run(['sed', '-i', f's/task: .*/task: {ENV_NAME}/', '/content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml'])
print(f"✅ 已設定環境: {ENV_NAME}")

In [ ]:
# === 選擇環境：Walker2d ===
ENV_NAME = "walker2d-medium-expert-v2"

import subprocess
subprocess.run(['sed', '-i', f's/task: .*/task: {ENV_NAME}/', '/content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml'])
print(f"✅ 已設定環境: {ENV_NAME}")

In [ ]:
# === 選擇環境：Hopper ===
ENV_NAME = "hopper-medium-expert-v2"

import subprocess
subprocess.run(['sed', '-i', f's/task: .*/task: {ENV_NAME}/', '/content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml'])
print(f"✅ 已設定環境: {ENV_NAME}")

In [ ]:
# === 執行訓練（100K 步）===
# 會自動：
# 1. 從 HuggingFace 下載之前的 loss_log.json（接續圖表）
# 2. 每 1000 步上傳 checkpoint 和圖表到 HuggingFace
# 3. 只保留 1000, 6000, 11000, 16000... 步的 checkpoint（間隔 5000 步）
#    這樣可以在推論時看到學習過程

!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/root/.mujoco/mujoco210/bin && python3 /content/run.py

In [ ]:
# === 確認 checkpoint 狀態（可選）===
import os
import json

checkpoint_dir = '/content/diffuserlite.github.io/results/diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2'
loss_log_path = '/content/diffuserlite.github.io/results/loss_log.json'

print("=== Checkpoint 狀態 ===")
if os.path.exists(checkpoint_dir):
    files = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith('.pt')])
    for f in files:
        filepath = os.path.join(checkpoint_dir, f)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  {f}: {size_mb:.1f} MB")
else:
    print("  (無 checkpoint)")

print("\n=== Loss Log 狀態 ===")
if os.path.exists(loss_log_path):
    with open(loss_log_path, 'r') as f:
        loss_log = json.load(f)
    if loss_log:
        print(f"  共 {len(loss_log)} 筆記錄")
        print(f"  最新步數: {loss_log[-1].get('gradient_steps', 0)}")
    else:
        print("  (空的)")
else:
    print("  (不存在)")

In [ ]:
# === 手動上傳到 HuggingFace（備份用，一般不需要執行）===
# 訓練腳本已自動每 1000 步上傳，這裡只是備份選項

from huggingface_hub import HfApi
import os
import glob

api = HfApi()
repo_id = "JoshuaLee0816/diffuserlite-results"

print("正在手動上傳到 Hugging Face...")

files_to_upload = [
    ('/content/diffuserlite.github.io/results/loss_log.json', 'loss_log.json'),
    ('/content/diffuserlite.github.io/results/figures/loss_curve.png', 'figures/loss_curve.png'),
]

checkpoint_dir = '/content/diffuserlite.github.io/results/diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2'
for name in ['diffusion0', 'diffusion1', 'diffusion2', 'invdyn']:
    ckpt_path = f'{checkpoint_dir}/{name}_ckpt_latest.pt'
    if os.path.exists(ckpt_path):
        files_to_upload.append((ckpt_path, f'diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2/{name}_ckpt_latest.pt'))

for local_path, repo_path in files_to_upload:
    if os.path.exists(local_path):
        try:
            api.upload_file(
                path_or_fileobj=local_path,
                path_in_repo=repo_path,
                repo_id=repo_id,
            )
            print(f"  ✅ {repo_path}")
        except Exception as e:
            print(f"  ❌ {repo_path}: {e}")

print(f"\n✅ 上傳完成！")
print(f"🔗 https://huggingface.co/{repo_id}")